# Dataset Extraction

This is a notebook for getting started with the data.

The dataset used is **SOLAQUA**, available from [SINTEF Open Data](https://data.sintef.no/feature/fe-a8f86232-5107-495e-a3dd-a86460eebef6).  


## Installing Packages

In [2]:
%pip install rosbags


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.9/723.9 kB 28.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [rosbags]m4/5 [rosbags]aml]clib]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from collections import defaultdict 
from rosbags.highlevel import AnyReader
from pathlib import Path
import numpy as np
import cv2
import re

## Defining .bag files

All data files should be placed in the `../data/SOLAQUA` folder.

- `*_data.bag` → contains **sensor data** (ROS bag format).
- `*_video.bag` → contains **video images** (ROS bag format).

The dataset used is **SOLAQUA**, available from [SINTEF Open Data](https://data.sintef.no/feature/fe-a8f86232-5107-495e-a3dd-a86460eebef6).  

In [2]:
# Change these two lines to switch dataset
DATA_BAG  = Path("../data/SOLAQUA/2024-08-20_13-55-34_data.bag")   # sensor data
VIDEO_BAG = Path("../data/SOLAQUA/2024-08-20_13-55-34_video.bag")  # camera and sonar video

# Output folder for extracted frames, videos, sonar arrays, etc.
OUT_ROOT = Path("../data/SOLAQUA/processed")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"Using data bag : {DATA_BAG.resolve()}")
print(f"Using video bag: {VIDEO_BAG.resolve()}")
print(f"Output folder : {OUT_ROOT.resolve()}")

Using data bag : /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/data/SOLAQUA/2024-08-20_13-55-34_data.bag
Using video bag: /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/data/SOLAQUA/2024-08-20_13-55-34_video.bag
Output folder : /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/data/SOLAQUA/processed


## List Topics

In [3]:
def human_hz(count, duration_s):
    if count == 0 or duration_s <= 0:
        return 0.0
    return count / duration_s

for bag in [DATA_BAG, VIDEO_BAG]:
    print(f"\n=== {bag.name} ===")
    if not bag.exists():
        print("  (missing)")
        continue

    counts = defaultdict(int)
    first_ts = defaultdict(lambda: None)
    last_ts  = defaultdict(lambda: None)
    types = {}

    with AnyReader([bag]) as r:
        for c in r.connections:
            types[c.topic] = c.msgtype
        for conn, ts, _ in r.messages():
            t = conn.topic
            counts[t] += 1
            if first_ts[t] is None or ts < first_ts[t]:
                first_ts[t] = ts
            if last_ts[t] is None or ts > last_ts[t]:
                last_ts[t] = ts

    if not counts:
        print("  (no messages)")
        continue

    col_topic = max(len(t) for t in counts.keys())
    col_type  = max(len(types.get(t, "")) for t in counts.keys())
    header = f"{'TOPIC'.ljust(col_topic)}  {'TYPE'.ljust(col_type)}  COUNT    START(ns)          END(ns)            DURATION(s)  ~HZ"
    print(header)
    print("-" * len(header))

    for t in sorted(counts.keys()):
        n = counts[t]
        t0 = first_ts[t]
        t1 = last_ts[t]
        dur_s = (t1 - t0) / 1e9 if (t0 is not None and t1 is not None) else 0.0
        hz = human_hz(n, dur_s)
        print(
            f"{t.ljust(col_topic)}  "
            f"{types.get(t,'').ljust(col_type)}  "
            f"{str(n).rjust(5)}    "
            f"{str(t0).rjust(16)}  "
            f"{str(t1).rjust(16)}  "
            f"{dur_s:11.3f}  {hz:5.2f}"
        )


=== 2024-08-20_13-55-34_data.bag ===
TOPIC                                     TYPE                                     COUNT    START(ns)          END(ns)            DURATION(s)  ~HZ
--------------------------------------------------------------------------------------------------------------------------------------------------
/bluerov2/alive                           std_msgs/msg/Float32                        99    1724154937968690000  1724154986552768700       48.584   2.04
/bluerov2/armed                           std_msgs/msg/Float32                       100    1724154937115084400  1724154986552842300       49.438   2.02
/bluerov2/battery                         messages/msg/BatteryStatus                  44    1724154941243289500  1724154986201058200       44.958   0.98
/bluerov2/modes                           joystick/msg/ModeManager2                    1    1724154939567437000  1724154939567437000        0.000   0.00
/commanded_thrust                         rospy_tutorial

## Extracting data

### Extract frames for _video.bag 

Remeber to handle if _video.bag has compressed and uncomressed images. 

Remember to handle if we have multiple topics under the same type, we will get flickering effect in video and "duplicate" images. 

In [4]:
# Extract camera frames from the selected VIDEO_BAG
# New layout (per-topic):
#   ./output/<bag_timestamp>/camera/<topic_sanitized>/image_frames/<topic_sanitized>_<ros_timestamp>.jpg
#
# - Derives the timestamp folder from the bag (strips "_video" suffix).
# - Saves frames for each image topic into its own subfolder to avoid interleaving/flicker.
# - Handles CompressedImage and common raw Image encodings.
# - Adds per-topic stats and progress logs every 100 saved frames.

from pathlib import Path
import numpy as np
import cv2

# Assumes these are already defined in your environment:
# VIDEO_BAG: Path to the .bag
# OUT_ROOT: base output Path
# AnyReader: bag reader (e.g., rosbags, mcap, etc.)

# --- Config ---
# If you want to include only certain topics, put them here; otherwise leave as None to include all image topics.
TOPIC_INCLUDE = None  # e.g., ["/image/compressed_image/data", "/ted/image"]

# If you want to exclude certain topics, list them here (checked after include).
TOPIC_EXCLUDE = []    # e.g., ["/image/compressed_image/camera_info"]

# --- Output scaffolding ---
bag_stem = VIDEO_BAG.stem.replace("_video", "")
RUN_ROOT = OUT_ROOT / bag_stem / "camera"
RUN_ROOT.mkdir(parents=True, exist_ok=True)

assert VIDEO_BAG.exists(), f"Missing video bag: {VIDEO_BAG}"

print(f"[INFO] Reading {VIDEO_BAG.name}")
print(f"[INFO] Saving frames under: {RUN_ROOT}")

def sanitize_topic(topic: str) -> str:
    """Make a filesystem-safe topic label (stable and readable)."""
    # Strip leading slash, replace remaining slashes with double underscores
    # Keep alphanum, underscore and dash, map others to underscore.
    base = topic.strip("/")
    safe = base.replace("/", "__")
    safe = "".join(ch if (ch.isalnum() or ch in ("_", "-", ".", "__")) else "_" for ch in safe)
    return safe or "topic"

def ensure_topic_dirs(topic: str, cache: dict) -> Path:
    """Create and cache the per-topic image_frames directory."""
    if topic not in cache:
        safe = sanitize_topic(topic)
        topic_dir = RUN_ROOT / safe / "image_frames"
        topic_dir.mkdir(parents=True, exist_ok=True)
        cache[topic] = topic_dir
        print(f"[INFO] → Topic '{topic}' → {topic_dir}")
    return cache[topic]

def decode_raw_image(msg):
    """Decode sensor_msgs/msg/Image to BGR np.ndarray (uint8)."""
    h, w, step = msg.height, msg.width, msg.step
    enc = (msg.encoding or "").lower()
    buf = np.frombuffer(msg.data, dtype=np.uint8)

    # Common 8-bit encodings
    if enc in ("bgr8",):
        frame = buf.reshape(h, step)[:, :w*3].reshape(h, w, 3)
        return frame
    if enc in ("rgb8",):
        frame = buf.reshape(h, step)[:, :w*3].reshape(h, w, 3)
        return cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    if enc in ("mono8", "8uc1", "8uc1c1", "mono"):
        frame = buf.reshape(h, step)[:, :w]
        return cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    if enc in ("yuv422", "yuyv", "yuyv422", "yuv422_yuy2"):
        # 2 bytes per pixel
        frame = buf.reshape(h, step)[:, :w*2]
        return cv2.cvtColor(frame, cv2.COLOR_YUV2BGR_YUY2)

    # Fallback heuristic (can be wrong for Bayer/16-bit)
    chans = 3 if (step % w != 0) else max(1, step // w)
    try:
        raw = buf.reshape(h, step)[:, :w*chans].reshape(h, w, chans)
    except Exception:
        return None

    if chans == 1:
        return cv2.cvtColor(raw, cv2.COLOR_GRAY2BGR)
    if enc == "rgb8":  # just in case encoding was weirdly reported
        return cv2.cvtColor(raw, cv2.COLOR_RGB2BGR)
    return raw

def save_frame(topic_dir: Path, topic_safe: str, ts_ns: int, frame_bgr: np.ndarray):
    """Save a BGR frame as JPEG with topic+timestamp-based filename."""
    out = topic_dir / f"{topic_safe}_{ts_ns}.jpg"
    cv2.imwrite(str(out), frame_bgr)

# --- Main read loop (per-topic saving) ---
saved_by_topic = {}
skipped_by_topic = {}
topic_dirs_cache = {}

from contextlib import ExitStack
with ExitStack() as stack:
    r = stack.enter_context(AnyReader([VIDEO_BAG]))

    for i, (conn, ts, raw) in enumerate(r.messages()):
        msgtype = conn.msgtype
        topic = conn.topic

        # Filter for image-like topics only (skip CameraInfo, sonar custom msgs, etc.)
        if msgtype not in ("sensor_msgs/msg/CompressedImage", "sensor_msgs/msg/Image"):
            continue

        if TOPIC_INCLUDE and topic not in TOPIC_INCLUDE:
            continue
        if TOPIC_EXCLUDE and topic in TOPIC_EXCLUDE:
            continue

        topic_dir = ensure_topic_dirs(topic, topic_dirs_cache)
        topic_safe = topic_dir.parent.name  # the sanitized topic folder name

        # Decode
        if msgtype == "sensor_msgs/msg/CompressedImage":
            msg = r.deserialize(raw, msgtype)
            arr = np.frombuffer(msg.data, np.uint8)
            frame = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        else:  # sensor_msgs/msg/Image
            msg = r.deserialize(raw, msgtype)
            frame = decode_raw_image(msg)

        # Count bookkeeping
        if topic not in saved_by_topic:
            saved_by_topic[topic] = 0
            skipped_by_topic[topic] = 0

        if frame is None:
            skipped_by_topic[topic] += 1
            continue

        save_frame(topic_dir, topic_safe, ts, frame)
        saved_by_topic[topic] += 1

        # Per-topic progress
        if saved_by_topic[topic] % 100 == 0:
            print(f"[INFO] [{topic}] Saved {saved_by_topic[topic]} frames …")

# --- Summary ---
print("\n[DONE] Per-topic results:")
total_saved = 0
total_skipped = 0
for t in sorted(saved_by_topic.keys()):
    s = saved_by_topic[t]
    k = skipped_by_topic.get(t, 0)
    total_saved += s
    total_skipped += k
    safe = sanitize_topic(t)
    out_dir = RUN_ROOT / safe / "image_frames"
    print(f"  - {t} → saved: {s:5d}, skipped: {k:5d}, dir: {out_dir}")

print(f"\n[TOTAL] Saved {total_saved} frames across {len(saved_by_topic)} topics.")
if total_skipped:
    print(f"[WARN] Skipped {total_skipped} frames (decode failures).")


[INFO] Reading 2024-08-20_13-55-34_video.bag
[INFO] Saving frames under: ../data/SOLAQUA/processed/2024-08-20_13-55-34/camera
[INFO] → Topic '/ted/image' → ../data/SOLAQUA/processed/2024-08-20_13-55-34/camera/ted__image/image_frames
[INFO] → Topic '/image/compressed_image/data' → ../data/SOLAQUA/processed/2024-08-20_13-55-34/camera/image__compressed_image__data/image_frames
[INFO] [/ted/image] Saved 100 frames …
[INFO] [/image/compressed_image/data] Saved 100 frames …
[INFO] [/ted/image] Saved 200 frames …
[INFO] [/image/compressed_image/data] Saved 200 frames …
[INFO] [/ted/image] Saved 300 frames …
[INFO] [/image/compressed_image/data] Saved 300 frames …
[INFO] [/ted/image] Saved 400 frames …
[INFO] [/image/compressed_image/data] Saved 400 frames …
[INFO] [/ted/image] Saved 500 frames …
[INFO] [/image/compressed_image/data] Saved 500 frames …
[INFO] [/ted/image] Saved 600 frames …
[INFO] [/image/compressed_image/data] Saved 600 frames …
[INFO] [/ted/image] Saved 700 frames …
[INFO] [

### Make MP4 for _video.bag


In [21]:
from pathlib import Path
import re
import cv2

# Build MP4 from frames in ./output/<timestamp>/camera/<topic_safe>/image_frames/
# Save as    ./output/<timestamp>/camera/<timestamp>.mp4

bag_stem   = VIDEO_BAG.stem.replace("_video", "")
RUN_ROOT   = OUT_ROOT / bag_stem / "camera"

# choose the topic you want to render:
TOPIC_SAFE = "image__compressed_image__data"   # e.g., "image__compressed_image__data" or "ted__image"
FRAMES_DIR = RUN_ROOT / TOPIC_SAFE / "image_frames"
OUT_MP4    = RUN_ROOT / f"{bag_stem + TOPIC_SAFE}.mp4"

assert FRAMES_DIR.exists(), f"Frame folder not found: {FRAMES_DIR}. Extract frames first."

# files look like: <topic_safe>_<ros_timestamp>.jpg
pat = re.compile(rf"^{re.escape(TOPIC_SAFE)}_(\d+)\.jpg$")
def ts_from_name(p: Path) -> int:
    m = pat.match(p.name)
    return int(m.group(1)) if m else -1

frames = [p for p in FRAMES_DIR.glob(f"{TOPIC_SAFE}_*.jpg") if pat.match(p.name)]
frames.sort(key=ts_from_name)
assert frames, f"No frames found in {FRAMES_DIR} matching {TOPIC_SAFE}_*.jpg"

# Estimate capture FPS from timestamps (max possible)
ts_list = [ts_from_name(p) for p in frames]
dur_s = (ts_list[-1] - ts_list[0]) / 1e9 if len(ts_list) > 1 else 0.0
fps_est = (len(ts_list) / dur_s) if dur_s > 0 else 25.0
FPS = round(fps_est, 2)

# Video dimensions from first frame
first = cv2.imread(str(frames[0]))
assert first is not None, f"Failed to read first frame: {frames[0]}"
h, w = first.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # use 'avc1' if available for H.264
vw = cv2.VideoWriter(str(OUT_MP4), fourcc, FPS, (w, h))
assert vw.isOpened(), "VideoWriter failed to open. Check codec availability."

print(f"[INFO] Writing {len(frames)} frames → {OUT_MP4}")
print(f"[INFO] FPS={FPS}  size={w}x{h}")

written = 0
for i, fp in enumerate(frames, 1):
    img = cv2.imread(str(fp))
    if img is None:
        continue
    if img.shape[:2] != (h, w):
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
    vw.write(img)
    written += 1
    if written % 100 == 0:
        print(f"[INFO] Wrote {written}/{len(frames)} frames …")

vw.release()
print(f"[DONE] MP4 saved: {OUT_MP4}  ({written} frames at {FPS} FPS)")


[INFO] Writing 1997 frames → ../data/SOLAQUA/processed/2024-08-20_13-57-42/camera/2024-08-20_13-57-42image__compressed_image__data.mp4
[INFO] FPS=25.01  size=1280x720
[INFO] Wrote 100/1997 frames …
[INFO] Wrote 200/1997 frames …
[INFO] Wrote 300/1997 frames …
[INFO] Wrote 400/1997 frames …
[INFO] Wrote 500/1997 frames …
[INFO] Wrote 600/1997 frames …
[INFO] Wrote 700/1997 frames …
[INFO] Wrote 800/1997 frames …
[INFO] Wrote 900/1997 frames …
[INFO] Wrote 1000/1997 frames …
[INFO] Wrote 1100/1997 frames …
[INFO] Wrote 1200/1997 frames …
[INFO] Wrote 1300/1997 frames …
[INFO] Wrote 1400/1997 frames …
[INFO] Wrote 1500/1997 frames …
[INFO] Wrote 1600/1997 frames …
[INFO] Wrote 1700/1997 frames …
[INFO] Wrote 1800/1997 frames …
[INFO] Wrote 1900/1997 frames …
[DONE] MP4 saved: ../data/SOLAQUA/processed/2024-08-20_13-57-42/camera/2024-08-20_13-57-42image__compressed_image__data.mp4  (1997 frames at 25.01 FPS)


### Extract raw sonar frames to .npy 
This is for later!!

In [6]:
# Extract raw SonoptixECHO pings from VIDEO_BAG to .npy (exact float32 arrays)
# Output:
#   ./output/<timestamp>/echo/raw_frames/sonar_<ros_ts>.npy



assert VIDEO_BAG.exists(), f"Missing video bag: {VIDEO_BAG}"

# Derive <timestamp> folder from VIDEO_BAG (strip "_video")
bag_stem = VIDEO_BAG.stem.replace("_video", "")

ECHO_ROOT = OUT_ROOT / bag_stem / "echo"
RAW_DIR   = ECHO_ROOT / "raw_frames"
RAW_DIR.mkdir(parents=True, exist_ok=True)

saved_raw = 0
skipped   = 0

print(f"[INFO] Reading {VIDEO_BAG.name}")
print(f"[INFO] RAW out: {RAW_DIR}")

with AnyReader([VIDEO_BAG]) as r:
    for i, (conn, ts, raw) in enumerate(r.messages()):
        if conn.msgtype != "sensors/msg/SonoptixECHO":
            continue

        msg  = r.deserialize(raw, conn.msgtype)
        data = np.asarray(msg.array_data.data, dtype=np.float32)

        # Determine H, W from layout (fallback heuristics)
        dims = msg.array_data.layout.dim
        H = int(dims[0].size) if len(dims) > 0 else 1024
        W = int(dims[1].size) if len(dims) > 1 else 256

        if data.size != H * W:
            if data.size == 1024 * 256:
                H, W = 1024, 256
            elif data.size == 256 * 1024:
                H, W = 256, 1024
            else:
                print(f"[WARN] size mismatch: vec={data.size}, layout={H}x{W}; skip ts={ts}")
                skipped += 1
                continue

        sonar_raw = data.reshape(H, W)
        np.save(RAW_DIR / f"sonar_{ts}.npy", sonar_raw)
        saved_raw += 1

        if saved_raw % 50 == 0:
            finite = np.isfinite(sonar_raw)
            if np.any(finite):
                mn = float(np.min(sonar_raw[finite]))
                mx = float(np.max(sonar_raw[finite]))
                print(f"[INFO] {saved_raw} raw saved … shape={H}x{W}, min..max={mn:.3g}..{mx:.3g}")
            else:
                print(f"[INFO] {saved_raw} raw saved … (no finite values)")

print(f"[DONE] RAW frames: {saved_raw}  → {RAW_DIR}")
if skipped:
    print(f"[WARN] Skipped frames: {skipped}")


[INFO] Reading 2024-08-20_17-22-40_video.bag
[INFO] RAW out: output/2024-08-20_17-22-40/echo/raw_frames
[INFO] 50 raw saved … shape=1024x256, min..max=0..64
[INFO] 100 raw saved … shape=1024x256, min..max=0..64
[INFO] 150 raw saved … shape=1024x256, min..max=0..64
[INFO] 200 raw saved … shape=1024x256, min..max=0..64
[INFO] 250 raw saved … shape=1024x256, min..max=0..64
[INFO] 300 raw saved … shape=1024x256, min..max=0..62
[INFO] 350 raw saved … shape=1024x256, min..max=0..64
[INFO] 400 raw saved … shape=1024x256, min..max=0..63
[INFO] 450 raw saved … shape=1024x256, min..max=0..63
[INFO] 500 raw saved … shape=1024x256, min..max=0..62
[INFO] 550 raw saved … shape=1024x256, min..max=0..62
[INFO] 600 raw saved … shape=1024x256, min..max=0..62
[INFO] 650 raw saved … shape=1024x256, min..max=0..63
[INFO] 700 raw saved … shape=1024x256, min..max=0..62
[INFO] 750 raw saved … shape=1024x256, min..max=0..63
[DONE] RAW frames: 795  → output/2024-08-20_17-22-40/echo/raw_frames


In [7]:
# Create preview PNGs from saved sonar .npy frames (visualization only)
# Input : ./output/<timestamp>/echo/raw_frames/sonar_<ros_ts>.npy
# Output: ./output/<timestamp>/echo/quicklook/sonar_<ros_ts>.png

# Use the same timestamp folder as above (from VIDEO_BAG)
bag_stem    = VIDEO_BAG.stem.replace("_video", "")
ECHO_ROOT   = OUT_ROOT / bag_stem / "echo"
RAW_DIR     = ECHO_ROOT / "raw_frames"
PREVIEW_DIR = ECHO_ROOT / "quicklook"
PREVIEW_DIR.mkdir(parents=True, exist_ok=True)

assert RAW_DIR.exists(), f"Raw frames folder not found: {RAW_DIR}"

# --- Preview normalization mode ---
# 'clip01': assumes data roughly in [0,1] and clips outside
# 'p01'   : per-frame 1st..99th percentile stretch (more legible)
PREVIEW_MODE = "p01"

def preview_u8(frame: np.ndarray) -> np.ndarray:
    v = frame.astype(np.float32)
    if PREVIEW_MODE == "clip01":
        v = np.clip(v, 0.0, 1.0)
        v = (v * 255.0).astype(np.uint8)
    else:
        finite = np.isfinite(v)
        if not np.any(finite):
            return np.zeros_like(v, dtype=np.uint8)
        lo, hi = np.percentile(v[finite], [1.0, 99.0])
        if hi <= lo:
            hi = lo + 1e-6
        v = np.clip((v - lo) / (hi - lo), 0.0, 1.0)
        v = (v * 255.0).astype(np.uint8)
    # Optional transpose for display
    return v.T

# Sort by ROS timestamp in filename
re_ts = re.compile(r"sonar_(\d+)\.npy$")
def npy_key(p: Path):
    m = re_ts.search(p.name)
    return int(m.group(1)) if m else 0

npy_files = sorted(RAW_DIR.glob("sonar_*.npy"), key=npy_key)
assert npy_files, f"No .npy frames found in {RAW_DIR}"

saved_png = 0
for i, npy_path in enumerate(npy_files, 1):
    arr = np.load(npy_path)
    png = preview_u8(arr)
    out = PREVIEW_DIR / (npy_path.stem + ".png")
    cv2.imwrite(str(out), png)
    saved_png += 1
    if saved_png % 50 == 0:
        print(f"[INFO] Wrote {saved_png} previews …")

print(f"[DONE] Quicklook PNGs: {saved_png} → {PREVIEW_DIR}")
print(f"[INFO] Preview mode: {PREVIEW_MODE}")


[INFO] Wrote 50 previews …
[INFO] Wrote 100 previews …
[INFO] Wrote 150 previews …
[INFO] Wrote 200 previews …
[INFO] Wrote 250 previews …
[INFO] Wrote 300 previews …
[INFO] Wrote 350 previews …
[INFO] Wrote 400 previews …
[INFO] Wrote 450 previews …
[INFO] Wrote 500 previews …
[INFO] Wrote 550 previews …
[INFO] Wrote 600 previews …
[INFO] Wrote 650 previews …
[INFO] Wrote 700 previews …
[INFO] Wrote 750 previews …
[DONE] Quicklook PNGs: 795 → output/2024-08-20_17-22-40/echo/quicklook
[INFO] Preview mode: p01


# Evaluation

# NTNU_fishdata trained, SOLAQUA tested

### Eval


In [22]:
from ultralytics import YOLO
import subprocess

model = YOLO("../runs_uwyolo/fishdata_adamw150_2gpu/weights/best.pt")

results = model.predict(
    source="../data/SOLAQUA/processed/2024-08-20_13-57-42/camera/2024-08-20_13-57-42image__compressed_image__data.mp4",   
    imgsz=640,
    conf=0.25,
    iou=0.7,
    save=True,
    project="../runs_uwyolo",
    name="EVAL_fishdata_on_solaqua",
    exist_ok=True,
    save_txt=False,
    save_conf=False
)

# results is a list; use the first element
save_dir = results[0].save_dir
print("✅ Inference done. Results saved in:", save_dir)





WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1997) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/notebooks/../data/SOLAQUA/processed/2024-08-20_13-57-42/camera/2024-08-20_13-57-42image__compressed_image__data.mp4: 384x640 (no detections), 89.1ms
video 1/1 (frame 2/1997) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/notebooks/../data/SOLAQUA/processed/2024-08-20_13-57-42/camera/2024-08-20_13-57-42image__compressed_image__data.mp4: 384x640 (no detections), 12.3ms
video 1/1 (frame 3/1

### .avi -> mp4

In [23]:
import cv2
from pathlib import Path

# ==== CONFIG ====
input_path = Path("../runs_uwyolo/EVAL_fishdata_on_solaqua/2024-08-20_13-57-42image__compressed_image__data.avi")

output_path = input_path.with_suffix(".mp4")   # output with same name but .mp4

# ==== OPEN INPUT ====
cap = cv2.VideoCapture(str(input_path))
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video: {input_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")   # or 'avc1' for H.264 codec

# ==== OPEN OUTPUT ====
out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

print(f"🎞️ Converting {input_path.name} → {output_path.name} ({width}x{height} @ {fps:.2f}fps) ...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)

cap.release()
out.release()

print(f"✅ Done! Saved to: {output_path.resolve()}")


🎞️ Converting 2024-08-20_13-57-42image__compressed_image__data.avi → 2024-08-20_13-57-42image__compressed_image__data.mp4 (1280x720 @ 25.00fps) ...
✅ Done! Saved to: /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/runs_uwyolo/EVAL_fishdata_on_solaqua/2024-08-20_13-57-42image__compressed_image__data.mp4
